Imports

In [1]:
import datacube
import pandas as pd
import xarray as xr
import datetime
import utils.data_cube_utilities.data_access_api as dc_api  

import math
import numpy as np
import configparser

functions

In [210]:
def _normalized_indexes_calculator(array, stationLongitude, stationLatitude):

    for i in range(array.dims['time']-1,-1,-1):
        a = array.isel(time=i)
        if not math.isnan(a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").NDVI.values.mean()):
            ndvi = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").NDVI.values.mean()
            ndwi = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").NDWI.values.mean()
            ndmi = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").NDMI.values.mean()
            ndbi = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").NDBI.values.mean()
            
            ndviStd = a.NDVI.std().values + 0
            ndwiStd = a.NDWI.std().values + 0
            ndmiStd = a.NDMI.std().values + 0
            ndbiStd = a.NDBI.std().values + 0
            
            ndviMean = a.NDVI.mean().values + 0
            ndwiMean = a.NDWI.mean().values + 0
            ndmiMean = a.NDMI.mean().values + 0
            ndbiMean = a.NDBI.mean().values + 0
            
            date = a['time']
#             break
            return(ndvi, ndwi, ndmi, ndbi, ndviStd, ndwiStd, ndmiStd, ndbiStd, ndviMean, ndwiMean, ndmiMean, ndbiMean, date)

#     return(ndvi, ndwi, ndmi, ndbi, ndviStd, ndwiStd, ndmiStd, ndbiStd, ndviMean, ndwiMean, ndmiMean, ndbiMean, date)
    return(None, None, None, None, None, None, None, None, None, None, None, None, None)
    
def _land_surface_day_temperature_calculator(array, stationLongitude, stationLatitude):
    for i in range(array.dims['time']-1,-1,-1):
        a = array.isel(time=i)
        if not math.isnan(a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").LST_Day.values.mean()):
           
            lstDay = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").LST_Day.values.mean()
            date = a['time']
#             break
#     return(lstDay, date)
            return(lstDay, date)
    return(None, None)


def _land_surface_night_temperature_calculator(array, stationLongitude, stationLatitude):
    for i in range(array.dims['time']-1,-1,-1):
        a = array.isel(time=i)
        if not math.isnan(a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").LST_Night.values.mean()):
           
            lstNight = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").LST_Night.values.mean()
            date = a['time']
#             break
#     return(lstNight, date)
            return(lstNight, date)
    return(None, None)

In [175]:
#     for i in range(concatenatedSorted.dims['time']-1,-1,-1):
#         print("i", i)
# #         print(concatenatedSorted.isel(time=i).time.values)
#         print(concatenatedSorted.isel(time=i))        
# #         print()

i 3
<xarray.Dataset>
Dimensions:    (latitude: 3, longitude: 3)
Coordinates:
    time       datetime64[ns] 2020-05-18
  * latitude   (latitude) float64 45.6 45.61 45.61
  * longitude  (longitude) float64 12.87 12.88 12.88
Data variables:
    NDVI       (latitude, longitude) float64 nan nan nan nan nan nan nan nan nan
    NDWI       (latitude, longitude) float64 nan nan nan nan nan nan nan nan nan
    NDMI       (latitude, longitude) float64 nan nan nan nan nan nan nan nan nan
    NDBI       (latitude, longitude) float64 nan nan nan nan nan nan nan nan nan
Attributes:
    crs:      GEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["...
i 2
<xarray.Dataset>
Dimensions:    (latitude: 3, longitude: 3)
Coordinates:
    time       datetime64[ns] 2020-05-02
  * latitude   (latitude) float64 45.6 45.61 45.61
  * longitude  (longitude) float64 12.87 12.88 12.88
Data variables:
    NDVI       (latitude, longitude) float64 0.1728 0.2604 ... 0.2965 0.386
    NDWI       (latitude, longi

Product definitions

In [107]:
dc = datacube.Datacube(app='notebook_name')
products_info = dc.list_products()
products_info

,name,description,lat,label,lon,format,creation_time,instrument,product_type,platform,time,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,
24,ls7_level2_scene,Landsat 7 Level 1 Collection-1,None,None,None,GeoTIFF,None,ETM+,level1,LANDSAT_7,None,"GEOGCRS[""WGS 84"",DATUM[""World Geodetic System ...","[0.004491576420598, 0.004491576420598]",None,"(latitude, longitude)"
25,ls8_level1_scene,Landsat 8 Level 1 Tier 1,None,None,None,GeoTIFF,None,OLI_TIRS,level1,LANDSAT_8,None,"GEOGCRS[""WGS 84"",DATUM[""World Geodetic System ...","[0.004491576420598, 0.004491576420598]",None,"(latitude, longitude)"
26,mod11a1_006_v6,Terra Land Surface Temperature and Emissivity ...,None,None,None,GeoTIFF,None,Terra,LST&E,MODIS,None,"GEOGCRS[""WGS 84"",DATUM[""World Geodetic System ...","[0.004491576420598, 0.004491576420598]",None,"(latitude, longitude)"
17,sentinel2_level1C_scene,Sentinel 2 Level 1C MSI:Multispectral Instrument,None,None,None,GeoTIFF,None,MSI,level1C,SENTINEL_2,None,"GEOGCRS[""WGS 84"",DATUM[""World Geodetic System ...","[0.004491576420598, 0.004491576420598]",None,"(latitude, longitude)"


Read the dataframe

In [109]:
dataframe = pd.read_csv("/home/datacube-epidemics/Datacube/stations/WNV-Culex-Veneto-2020_static.csv")

In [110]:
dataframe.head()

,stationid,dt_placeme,WC_L_1km,W_area_1km,DEM_1000,Aspect_1000,Slope_1000,Coast_dist_1000,WC_dist_1000,Flow_acc_1000,x,y
0,1,2020-05-18,6209.745276,188580.173358,2.0,181.852737,3.170475,755.338440,181.0,0.768354,12.877089,45.606292
1,397,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.897642,45.351147
2,392,2020-05-18,4061.865740,38393.816264,0.0,171.079849,1.435196,6427.323730,37.0,3.974069,12.223217,45.510817
3,198,2020-05-18,7207.803820,604526.916814,5.0,221.362305,4.301568,24714.152344,203.0,60211.200000,12.797359,45.826409
4,199,2020-05-18,4721.254625,290379.143468,0.0,23.073887,4.410034,9295.942383,54.0,371.752900,12.859687,45.690322


feature engineering

In [225]:
dataframe['ending_date'] = pd.to_datetime(dataframe['dt_placeme'])-datetime.timedelta(days=30)

In [ ]:
dataframe['year'] = pd.DatetimeIndex(dataframe['dt_placeme']).year

In [271]:
headDataframe = dataframe.head(15)

In [272]:
headDataframe

,stationid,dt_placeme,WC_L_1km,W_area_1km,DEM_1000,Aspect_1000,Slope_1000,Coast_dist_1000,WC_dist_1000,Flow_acc_1000,x,y,ending_date,year
0,1,2020-05-18,6209.745276,188580.173358,2.0,181.852737,3.170475,755.338440,181.0,0.768354,12.877089,45.606292,2020-04-18,2020
1,397,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.897642,45.351147,2020-04-18,2020
2,392,2020-05-18,4061.865740,38393.816264,0.0,171.079849,1.435196,6427.323730,37.0,3.974069,12.223217,45.510817,2020-04-18,2020
3,198,2020-05-18,7207.803820,604526.916814,5.0,221.362305,4.301568,24714.152344,203.0,60211.200000,12.797359,45.826409,2020-04-18,2020
4,199,2020-05-18,4721.254625,290379.143468,0.0,23.073887,4.410034,9295.942383,54.0,371.752900,12.859687,45.690322,2020-04-18,2020
5,314,2020-05-18,1245.851777,68348.777425,5.0,215.254852,2.136426,12063.803711,881.0,54.745890,12.260767,45.594461,2020-04-18,2020
6,3,2020-05-18,5799.351520,107143.441344,0.0,159.036743,3.263651,2522.510010,311.0,4.224569,12.429368,45.558920,2020-04-18,2020
7,384,2020-05-18,4345.118169,43413.646995,0.0,201.230560,3.347325,3912.984863,375.0,92.658390,12.637082,45.544211,2020-04-18,2020
8,397,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.897642,45.351147,2020-04-18,2020
9,382,2020-05-18,9171.213029,10008.308274,8.0,236.141006,2.204349,21181.453125,102.0,1.320860,11.958691,45.445442,2020-04-18,2020


In [228]:
latitude = (dataframe.y.min(), dataframe.y.max())
longitude = (dataframe.x.min(), dataframe.x.max())
print('Latitude_values: ', latitude,' Longitude_values: ', longitude)

Latitude_values:  (44.902271999999996, 45.864106903809)  Longitude_values:  (10.791083, 12.985235000000001)


Read .ini file

In [229]:
parser = configparser.ConfigParser()
parser.read("/home/datacube-epidemics/Datacube/jupyter_notebooks/data_cube_notebooks/notebooks/settings.ini")

['/home/datacube-epidemics/Datacube/jupyter_notebooks/data_cube_notebooks/notebooks/settings.ini']

In [230]:
sentinelName = parser.get("Sentinel2", "name")
sentinelProduct = parser.get("Sentinel2", "product")
sentinelPlatform = parser.get("Sentinel2", "platform")
sentinelInstrument = parser.get("Sentinel2", "instrument")

l7Name = parser.get("Landsat7", "name")
l7Product = parser.get("Landsat7", "product")
l7Platform = parser.get("Landsat7", "platform")
l7Instrument = parser.get("Landsat7", "instrument")

l8Name = parser.get("Landsat8", "name")
l8Product = parser.get("Landsat8", "product")
l8Platform = parser.get("Landsat8", "platform")
l8Instrument = parser.get("Landsat8", "instrument")

modisName = parser.get("Modis", "name")
modisProduct = parser.get("Modis", "product")
modisPlatform = parser.get("Modis", "platform")
modisInstrument = parser.get("Modis", "instrument")

resolution = float(parser.get("Products", "resolution"))

In [211]:
ndviList = []
ndwiList = []
ndmiList = []
ndbiList = []

ndviStdList = []
ndwiStdList = []
ndmiStdList = []
ndbiStdList = []

ndviMeanList = []
ndwiMeanList = []
ndmiMeanList = []
ndbiMeanList = []

lstDayList = []
lstNightList = []

# for row in dataframe.itertuples(index=False):
for row in headDataframe.itertuples(index=False):
    
#     get dataframe columns
    stationLongitude = row.x
    stationLatitude = row.y
    
    time = (row.ending_date, row.dt_placeme)

#     informational message
    print("stationid %s date %s" % (row.stationid, row.dt_placeme))
    print("station longitude %s, station latitude %s, date of placement %s" % (stationLongitude, stationLatitude, row.dt_placeme))
    
#     load data as an xarray
    landsat7 = dc.load(product=l7Name, platform=l7Platform,
            latitude=(stationLatitude-resolution, stationLatitude+resolution), 
            longitude=(stationLongitude-resolution, stationLongitude+resolution),
            time=time)
    
    landsat8 = dc.load(product=l8Name, platform=l8Platform,
            latitude=(stationLatitude-resolution, stationLatitude+resolution), 
            longitude=(stationLongitude-resolution, stationLongitude+resolution),
            time=time)
    
    sentinel = dc.load(product=l7Name, platform=l7Platform,
            latitude=(stationLatitude-resolution, stationLatitude+resolution), 
            longitude=(stationLongitude-resolution, stationLongitude+resolution),
            time=time)
    
    modis = dc.load(product=modisName, platform=modisPlatform,
            latitude=stationLatitude, longitude=stationLongitude, time=time)
    
#     xarray concatenation along time
    concatenated = xr.concat([landsat7,landsat8,sentinel],dim='time')
    concatenatedSorted = concatenated.sortby('time')

#     custom function for calculating the normalized indexes
    ndvi,ndwi,ndmi,ndbi, ndviStd, ndwiStd, ndmiStd, ndbiStd, ndviMean, ndwiMean, ndmiMean, ndbiMean, date = _normalized_indexes_calculator(concatenatedSorted, stationLongitude, stationLatitude)
    
    lstDay, lstDayDate = _land_surface_day_temperature_calculator(modis, stationLongitude, stationLatitude)
    
    lstNight, lstNightDate = _land_surface_night_temperature_calculator(modis, stationLongitude, stationLatitude)

#     informational messages
    timestamp = (date - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    acquisitionDate = datetime.datetime.utcfromtimestamp(timestamp)
    print("date", acquisitionDate)
    print("ndvi", ndvi)
    print("ndwi", ndwi)
    print("ndmi", ndmi)
    print("ndbi", ndbi)
    
    print("ndvi std", ndviStd)
    print("ndwi std", ndwiStd)
    print("ndmi std", ndmiStd)
    print("ndbi std", ndbiStd)
    
    print("ndvi mean", ndviMean)
    print("ndwi mean", ndwiMean)
    print("ndmi mean", ndmiMean)
    print("ndbi mean", ndbiMean)
    
    
    lstDayTimestamp = (lstDayDate - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    lstDayAcquisitionDate = datetime.datetime.utcfromtimestamp(lstDayTimestamp)
    print("date", lstDayAcquisitionDate)
    print("lst day", lstDay)
    
    lstNightTimestamp = (lstNightDate - np.datetime64('1970-01-01T00:00:00Z'))/np.timedelta64(1, 's')
    lstNightAcquisitionDate = datetime.datetime.utcfromtimestamp(lstNightTimestamp)
    print("date", lstNightAcquisitionDate)
    print("lst night", lstNight) 
    
#     list appendage
    ndviList.append(ndvi)
    ndwiList.append(ndwi)
    ndmiList.append(ndmi)
    ndbiList.append(ndbi)
    
    ndviStdList.append(ndviStd)
    ndwiStdList.append(ndwiStd)
    ndmiStdList.append(ndmiStd)
    ndbiStdList.append(ndbiStd)
    
    ndviMeanList.append(ndviMean)
    ndwiMeanList.append(ndwiMean)
    ndmiMeanList.append(ndmiMean)
    ndbiMeanList.append(ndbiMean)
    
    lstDayList.append(lstDay)
    lstNightList.append(lstNight)

stationid 1 date 2020-05-18
station longitude 12.8770891263046, station latitude 45.606292341247205, date of placement 2020-05-18


/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:63: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:87: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


date 2020-05-02 00:00:00
ndvi 0.29959633191057017
ndwi -0.31823313297867306
ndmi 0.013198070875098854
ndbi -0.013198070875098854
ndvi std 0.13958407747215248
ndwi std 0.05614947828009332
ndmi std 0.14220827791935745
ndbi std 0.14220827791935745
ndvi mean 0.35709621379744905
ndwi mean -0.24698789974241445
ndmi mean 0.11261881837283866
ndbi mean -0.11261881837283866
date 2020-05-17 00:00:00
lst day 14972.0
date 2020-05-09 00:00:00
lst night 14384.0
stationid 397 date 2020-05-18
station longitude 11.897642, station latitude 45.351147, date of placement 2020-05-18


/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:63: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:87: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


date 2020-05-02 00:00:00
ndvi 0.47677784260752876
ndwi -0.15962197144862736
ndmi 0.26928864840280203
ndbi -0.26928864840280203
ndvi std 0.1289737079698127
ndwi std 0.06484287584011388
ndmi std 0.12119809464409038
ndbi std 0.12119809464409038
ndvi mean 0.41625787489095417
ndwi mean -0.2171309951184934
ndmi mean 0.20147815872171995
ndbi mean -0.20147815872171995
date 2020-05-17 00:00:00
lst day 15269.0
date 2020-05-08 00:00:00
lst night 14232.0
stationid 392 date 2020-05-18
station longitude 12.223217, station latitude 45.510816999999996, date of placement 2020-05-18


/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:63: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:87: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


date 2020-04-24 00:00:00
ndvi 0.30084441087169456
ndwi -0.2595098707688363
ndmi 0.049928152963894516
ndbi -0.049928152963894516
ndvi std 0.06787420192050753
ndwi std 0.02453709699796644
ndmi std 0.04271605240178446
ndbi std 0.04271605240178446
ndvi mean 0.30821270832582415
ndwi mean -0.2596932235820908
ndmi mean 0.055472127856151764
ndbi mean -0.055472127856151764
date 2020-05-17 00:00:00
lst day 15192.0
date 2020-05-07 00:00:00
lst night 14352.0
stationid 198 date 2020-05-18
station longitude 12.79735868934, station latitude 45.8264089987247, date of placement 2020-05-18


/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:63: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:87: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


date 2020-05-02 00:00:00
ndvi 0.5784594760400552
ndwi -0.24789610353461056
ndmi 0.30968525880479353
ndbi -0.30968525880479353
ndvi std 0.147831982541831
ndwi std 0.18689204709645912
ndmi std 0.04434733118896261
ndbi std 0.04434733118896261
ndvi mean 0.4573357457282959
ndwi mean -0.11012258898529222
ndmi mean 0.27539937042773177
ndbi mean -0.27539937042773177
date 2020-05-17 00:00:00
lst day 15095.0
date 2020-05-17 00:00:00
lst night 14521.0
stationid 199 date 2020-05-18
station longitude 12.859687275474698, station latitude 45.6903223491131, date of placement 2020-05-18
date 2020-05-02 00:00:00
ndvi 0.3600841395279551
ndwi -0.22770161180953993
ndmi 0.13508088939771382
ndbi -0.13508088939771382
ndvi std 0.09606378386753547
ndwi std 0.05026835740288254
ndmi std 0.09131868786523693
ndbi std 0.09131868786523693
ndvi mean 0.31577730078964666
ndwi mean -0.22475822303889142
ndmi mean 0.08499501646726587
ndbi mean -0.08499501646726587
date 2020-05-17 00:00:00
lst day 15158.0
date 2020-05-09 00

/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:63: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:82: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:87: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


Append the newly created lists as columns to the .csv file

In [212]:
headDataframe["ndvi"] = ndviList
headDataframe["ndwi"] = ndwiList
headDataframe["ndmi"] = ndmiList
headDataframe["ndbi"] = ndbiList

headDataframe["ndvi_std"] = ndviStdList
headDataframe["ndwi_std"] = ndwiStdList
headDataframe["ndmi_std"] = ndmiStdList
headDataframe["ndbi_std"] = ndbiStdList

headDataframe["ndvi_mean"] = ndviMeanList
headDataframe["ndwi_mean"] = ndwiMeanList
headDataframe["ndmi_mean"] = ndmiMeanList
headDataframe["ndbi_mean"] = ndbiMeanList

headDataframe["lst_day"] = lstDayList
headDataframe["lst_night"] = lstNightList

/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

Check

In [268]:
headDataframe

,stationid,dt_placeme,WC_L_1km,W_area_1km,DEM_1000,Aspect_1000,Slope_1000,Coast_dist_1000,WC_dist_1000,Flow_acc_1000,x,y,ending_date
0,1,2020-05-18,6209.745276,188580.173358,2.0,181.852737,3.170475,755.338440,181.0,0.768354,12.877089,45.606292,2020-04-18
1,397,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.897642,45.351147,2020-04-18
2,392,2020-05-18,4061.865740,38393.816264,0.0,171.079849,1.435196,6427.323730,37.0,3.974069,12.223217,45.510817,2020-04-18
3,198,2020-05-18,7207.803820,604526.916814,5.0,221.362305,4.301568,24714.152344,203.0,60211.200000,12.797359,45.826409,2020-04-18
4,199,2020-05-18,4721.254625,290379.143468,0.0,23.073887,4.410034,9295.942383,54.0,371.752900,12.859687,45.690322,2020-04-18
5,314,2020-05-18,1245.851777,68348.777425,5.0,215.254852,2.136426,12063.803711,881.0,54.745890,12.260767,45.594461,2020-04-18
6,3,2020-05-18,5799.351520,107143.441344,0.0,159.036743,3.263651,2522.510010,311.0,4.224569,12.429368,45.558920,2020-04-18
7,384,2020-05-18,4345.118169,43413.646995,0.0,201.230560,3.347325,3912.984863,375.0,92.658390,12.637082,45.544211,2020-04-18
8,397,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.897642,45.351147,2020-04-18
9,382,2020-05-18,9171.213029,10008.308274,8.0,236.141006,2.204349,21181.453125,102.0,1.320860,11.958691,45.445442,2020-04-18


In [232]:
headDataframe.shape

(15, 13)

In [277]:
tempDataframe = headDataframe[['stationid', 'x', 'y', 'year']]

In [278]:
tempDataframe = tempDataframe.drop_duplicates()

In [279]:
tempDataframe.shape

(13, 4)

In [282]:
    
    
def functionz(array):
#     for i in range(array.dims['time']-1,-1,-1):
#         a = array.isel(time=i)
#         if not math.isnan(a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").LST_Day.values.mean()):
           
#             lstNight = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").LST_Day.values.mean()
#             date = a['time']
# #             break
# #     return(lstNight, date)
#             return(lstNight, date)
#     return(None, None)
    return array.LST_Day.mean().values + 0
# a.NDVI.mean().values + 0

In [270]:
# dataframe['year'] = pd.DatetimeIndex(dataframe['dt_placeme']).year

In [265]:
import calendar

In [288]:
january =  []
february = []
march = []
april = []

for row in tempDataframe.itertuples(index=False):
    
#     comment
    stationLongitude = row.x
    stationLatitude = row.y
    currentYear = row.year
#     currentDate = row.dt_placeme
#     currentDateTime = datetime.datetime.strptime(currentDate, '%Y-%m-%d')
#     currentYear = currentDateTime.year
    
#     comment
    januaryRange = calendar.monthrange(currentYear, 1)
    februaryRange = calendar.monthrange(currentYear, 2)
    marchRange = calendar.monthrange(currentYear, 3)
    aprilRange = calendar.monthrange(currentYear, 4)
    
#     comment
    januaryFirst = datetime.datetime(currentYear, 1, 1).strftime("%Y-%m-%d")
    januaryLast = datetime.datetime(currentYear, 1, januaryRange[1]).strftime("%Y-%m-%d")
    
    februaryFirst = datetime.datetime(currentYear, 2, 1).strftime("%Y-%m-%d")
    februaryLast = datetime.datetime(currentYear, 2, februaryRange[1]).strftime("%Y-%m-%d")
    
    marchFirst = datetime.datetime(currentYear, 3, 1).strftime("%Y-%m-%d")
    marchLast = datetime.datetime(currentYear, 3, marchRange[1]).strftime("%Y-%m-%d")
    
    aprilFirst = datetime.datetime(currentYear, 4, 1).strftime("%Y-%m-%d")
    aprilLast = datetime.datetime(currentYear, 4, aprilRange[1]).strftime("%Y-%m-%d")
    
#     comment
    januaryTime = (januaryFirst, januaryLast)
    februaryTime = (februaryFirst, februaryLast)
    marchTime = (marchFirst, marchLast)
    aprilTime = (aprilFirst, aprilLast)
    
#     comment
    modisJanuary = dc.load(product=modisName, platform=modisPlatform,
            latitude=stationLatitude, longitude=stationLongitude, time=januaryTime)
    
    modisFebruary = dc.load(product=modisName, platform=modisPlatform,
            latitude=stationLatitude, longitude=stationLongitude, time=februaryTime)
    
    modisMarch = dc.load(product=modisName, platform=modisPlatform,
            latitude=stationLatitude, longitude=stationLongitude, time=marchTime)
    
    modisApril = dc.load(product=modisName, platform=modisPlatform,
            latitude=stationLatitude, longitude=stationLongitude, time=aprilTime)
    
#     comment
    lstJanuary = functionz(modisJanuary)
    lstFebruary = functionz(modisFebruary)
    lstMarch = functionz(modisMarch)
    lstApril = functionz(modisApril)
    
#     comment
    january.append([row.stationid, currentYear, lstJanuary])
    february.append([row.stationid, currentYear, lstFebruary])
    march.append([row.stationid, currentYear, lstMarch])
    april.append([row.stationid, currentYear, lstApril])
    
#     debug
#     print("currentDate", currentDate)
#     print("firstDate", firstDate)
#     print("lastDate", lastDate)
#     print("currentDate TYPE", type(currentDate))
#     print("lastDate TYPE", type(lastDate))
#     print("firstDateStr", firstDateStr)
#     print("lastDateStr", lastDateStr)
#     print("--------------------")

In [291]:
januaryDataframe = pd.DataFrame(january, columns=["stationid", "year", "LST_Jan_mean"])

februaryDataframe = pd.DataFrame(february, columns=["stationid", "year", "LST_Feb_mean"])

marchDataframe = pd.DataFrame(march, columns=["stationid", "year", "LST_Mar_mean"])

aprilDataframe = pd.DataFrame(april, columns=["stationid", "year", "LST_Apr_mean"])

In [296]:
mergedDataframe = pd.merge(tempDataframe, januaryDataframe,  how='left', left_on=['stationid', 'year'], right_on = ['stationid', 'year'])
    
mergedDataframe = pd.merge(mergedDataframe, februaryDataframe,  how='left', left_on=['stationid', 'year'], right_on = ['stationid', 'year'])
mergedDataframe = pd.merge(mergedDataframe, marchDataframe,  how='left', left_on=['stationid', 'year'], right_on = ['stationid', 'year'])
mergedDataframe = pd.merge(mergedDataframe, aprilDataframe,  how='left', left_on=['stationid', 'year'], right_on = ['stationid', 'year'])

In [297]:
mergedDataframe

,stationid,x,y,year,LST_Jan_mean,LST_Feb_mean,LST_Mar_mean,LST_Apr_mean
0,1,12.877089,45.606292,2020,13936.833333,14228.700000,14358.181818,14980.500000
1,397,11.897642,45.351147,2020,13939.833333,14286.437500,14504.176471,15032.181818
2,392,12.223217,45.510817,2020,13990.800000,14299.866667,14536.461538,14994.900000
3,198,12.797359,45.826409,2020,13986.687500,14248.230769,14478.666667,14969.333333
4,199,12.859687,45.690322,2020,13950.666667,14242.928571,14484.071429,15068.647059
5,314,12.260767,45.594461,2020,13946.526316,14264.266667,14504.500000,14955.857143
6,3,12.429368,45.558920,2020,13949.444444,14298.916667,14450.428571,14867.818182
7,384,12.637082,45.544211,2020,13934.352941,14272.916667,14411.333333,14925.400000
8,382,11.958691,45.445442,2020,13951.235294,14294.533333,14532.000000,14969.652174
9,376,11.459718,45.701843,2020,14073.952381,14383.578947,14601.384615,14969.904762


In [293]:
januaryDataframe

,stationid,year,LST_Jan_mean
0,1,2020,13936.833333
1,397,2020,13939.833333
2,392,2020,13990.800000
3,198,2020,13986.687500
4,199,2020,13950.666667
5,314,2020,13946.526316
6,3,2020,13949.444444
7,384,2020,13934.352941
8,382,2020,13951.235294
9,376,2020,14073.952381


In [289]:
january

[[1, 2020, 13936.833333333334],
 [397, 2020, 13939.833333333334],
 [392, 2020, 13990.8],
 [198, 2020, 13986.6875],
 [199, 2020, 13950.666666666666],
 [314, 2020, 13946.526315789473],
 [3, 2020, 13949.444444444445],
 [384, 2020, 13934.35294117647],
 [382, 2020, 13951.235294117647],
 [376, 2020, 14073.952380952382],
 [373, 2020, 13983.3125],
 [346, 2020, 13956.4],
 [340, 2020, 13935.6875]]

In [168]:
# row
# concatenatedSorted
# ndviList

<xarray.Dataset>
Dimensions:    (latitude: 3, longitude: 3, time: 4)
Coordinates:
  * time       (time) datetime64[ns] 2020-04-24 2020-04-24 2020-05-02 2020-05-18
  * latitude   (latitude) float64 45.6 45.61 45.61
  * longitude  (longitude) float64 12.87 12.88 12.88
Data variables:
    NDVI       (time, latitude, longitude) float64 0.2413 0.2305 ... nan nan
    NDWI       (time, latitude, longitude) float64 -0.2409 -0.2638 ... nan nan
    NDMI       (time, latitude, longitude) float64 0.02101 -0.01646 ... nan nan
    NDBI       (time, latitude, longitude) float64 -0.02101 0.01646 ... nan nan
Attributes:
    crs:      GEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["...

In [191]:
# longitude=12.8770891263046

In [192]:
# time=('2020-05-10', '2020-05-20')

In [193]:
# latitude=45.6062923412472

In [195]:
# modis = dc.load(product=modisName, platform=modisPlatform,
#         latitude=latitude, longitude=longitude, time=time)    

In [136]:
# landsat7 = dc.load(product=sentinelName, platform=sentinelPlatform,
#             latitude=(latitude-resolution, latitude+resolution), longitude=(longitude-resolution, longitude+resolution),
#             time=time)

In [196]:
# modis

<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1, time: 11)
Coordinates:
  * time       (time) datetime64[ns] 2020-05-10 2020-05-11 ... 2020-05-20
  * latitude   (latitude) float64 45.61
  * longitude  (longitude) float64 12.88
Data variables:
    LST_Day    (time, latitude, longitude) float64 nan nan 1.49e+04 ... nan nan
    QC_Day     (time, latitude, longitude) float64 2.0 2.0 65.0 ... 2.0 2.0 2.0
    LST_Night  (time, latitude, longitude) float64 nan nan nan ... nan 1.433e+04
    QC_Night   (time, latitude, longitude) float64 2.0 2.0 2.0 ... 2.0 2.0 65.0
Attributes:
    crs:      GEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["...

In [219]:
# # a = landsat7.sel(time='2018-05-05')
# a = modis.sel(time='2020-05-17')
# a

<xarray.Dataset>
Dimensions:    (latitude: 1, longitude: 1)
Coordinates:
    time       datetime64[ns] 2020-05-17
  * latitude   (latitude) float64 45.69
  * longitude  (longitude) float64 12.86
Data variables:
    LST_Day    (latitude, longitude) float64 1.516e+04
    QC_Day     (latitude, longitude) float64 0.0
    LST_Night  (latitude, longitude) float64 nan
    QC_Night   (latitude, longitude) float64 2.0
Attributes:
    crs:      GEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["...

In [220]:
# # latitude, longitude
# lstDay = a.sel(latitude=stationLatitude, longitude=stationLongitude, method="nearest").LST_Day.values.mean()
# lstDay

15158.0

In [178]:
# a.sel(latitude=latitude, longitude=longitude, method="nearest").NDVI.values.mean()

0.21076233183856502

In [223]:
# longitude
# latitude
landsat7.time

<xarray.DataArray 'time' (time: 1)>
array(['2020-04-24T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-04-24
Attributes:
    units:    seconds since 1970-01-01 00:00:00

In [141]:
# a.sel(latitude=slice(latitude), longitude=slice(longitude)).NDVI.values
# 0.210762331838565

array([[0.12428473, 0.1350826 ],
       [0.27265356, 0.21076233]])

In [142]:
# landsat7.sel(latitude=slice(latitude), longitude=slice(longitude)).NDVI.mean()

In [181]:
# a.NDVI.std().values + 0

0.07365536685474336

In [144]:
# a.NDVI.mean().values

array(0.20336531)

In [145]:
# landsat7.sel(time=slice('2020-04-24')).NDVI.std().values

/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


array(nan)

In [146]:
# type(landsat7.sel(time=slice('2020-04-24')).NDVI.std().values)

/home/datacube-epidemics/miniconda3/envs/odc/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


numpy.ndarray